In [1]:
import torch
inputs = torch.tensor(
    [[
       [0.43, 0.15, 0.89], # Your     
       [0.55, 0.87, 0.66], # journey  
       [0.57, 0.85, 0.64], # starts   
       [0.22, 0.58, 0.33], # with     
       [0.77, 0.25, 0.10], # one      
       [0.05, 0.80, 0.55]  # step     
    ],
    [
       [0.43, 0.15, 0.89], # His     
       [0.55, 0.87, 0.66], # journey  
       [0.57, 0.85, 0.64], # end   
       [0.22, 0.58, 0.33], # at     
       [0.77, 0.25, 0.10], # last      
       [0.05, 0.80, 0.55]  # way     
    ]
])

In [2]:
inputs.shape

torch.Size([2, 6, 3])

In [3]:
import torch.nn as nn
class MaskedSelfAttention(nn.Module):
    def __init__(self,d_in,d_out, context_length, dropout, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in,d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in,d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in,d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length), diagonal=1)
        )
        
    def forward(self,x):
        no_of_input, no_of_token, embedding_dim = x.shape
        queries = self.W_query(x)
        keys = self.W_key(x)
        values = self.W_value(x)
        
        attention_scores = queries @ keys.transpose(1, 2)
        attention_scores = attention_scores.masked_fill(
            self.mask.bool()[:no_of_token , :no_of_token], -torch.inf
        )
        
        d_k = keys.shape[-1]
        attention_weights = torch.softmax(
            attention_scores / d_k**0.5 , dim=-1
        )
        attention_weights = self.dropout(attention_weights)
        
        context_vector = attention_weights @ values
        return context_vector
        

In [4]:
class MultiHeadAttentionWrapper(nn.Module):
    def __init__(self,d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        self.heads = nn.ModuleList(
            [MaskedSelfAttention(d_in,d_out,context_length,dropout,qkv_bias) for _ in range(num_heads)]
        )
        
    def forward(self,x):
        return torch.concat([head(x) for head in self.heads], dim=-1)

In [5]:
d_in, d_out = 3,2
context_length = inputs.shape[1]    # no_of_token

In [6]:
mha = MultiHeadAttentionWrapper(d_in,d_out,context_length,0.0,num_heads=2)

In [7]:
context_vector = mha(inputs)

In [8]:
context_vector

tensor([[[-0.2649,  0.5191,  0.0477,  0.2037],
         [-0.2144,  0.6240,  0.0334, -0.0273],
         [-0.1958,  0.6542,  0.0272, -0.0901],
         [-0.1696,  0.5858,  0.0262, -0.1163],
         [-0.1271,  0.5440,  0.0030, -0.0717],
         [-0.1365,  0.5346,  0.0144, -0.1197]],

        [[-0.2649,  0.5191,  0.0477,  0.2037],
         [-0.2144,  0.6240,  0.0334, -0.0273],
         [-0.1958,  0.6542,  0.0272, -0.0901],
         [-0.1696,  0.5858,  0.0262, -0.1163],
         [-0.1271,  0.5440,  0.0030, -0.0717],
         [-0.1365,  0.5346,  0.0144, -0.1197]]], grad_fn=<CatBackward0>)